In [ ]:
!pip install openai-whisper
!pip install zipfile-deflate64


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801360 sha256=baf07d652a923b481de71bbca0136fa528b14c89c42d2b9d2e147124432693db
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# unzip and pickle load 'all_embeddings0_2k.zip'

import zipfile_deflate64 as zipfile
import pickle
import os

if not os.path.isfile('data/all_embeddings0_2k.pkl'):
  with zipfile.ZipFile('drive/MyDrive/all_embeddings0_2k.zip', 'r') as zip_ref:
      zip_ref.extractall('data')
else:
  print('file already exists')

with open('data/all_embeddings0_2k.pkl', 'rb') as f:
    data = pickle.load(f)

file already exists


In [ ]:
import torch
import pickle
import numpy as np
import re
from whisper.normalizers import EnglishTextNormalizer
import editdistance

In [ ]:
data_type = 'embedding'
# data_type = 'logits'

In [ ]:
# with open('all_data.pkl','rb') as fp:
#   data = pickle.load(fp)

In [ ]:
vocab = np.array(data['vocab']+['eps'])
tags = data['tags']

# remove sample including numbers
tags = [tag for tag in tags if not re.search('\d',tag['transcription'])]

In [ ]:
def transcribe(logits,vocab):
  out = []
  tokens = vocab[logits.argmax(1)]
  prev_empty = True
  for token in tokens:
    empty = (token == vocab[-1])
    if not empty:
      if prev_empty:
        out.append(token)
      elif token != out[-1]:
        out.append(token)
    prev_empty = (token == vocab[-1])
  out = ''.join(out)
  out = re.sub(' +', ' ',out)
  return out

def eval_wer_cer(gt_char,pred_char):
  cer = editdistance.distance(gt_char,pred_char)/len(gt_char)
  gt = gt_char.split(' ')
  pred = pred_char.split(' ')
  wer = editdistance.distance(gt,pred)/len(gt)
  return wer,cer

In [ ]:
# total_pred = ' '.join([transcribe(tag['logits'],vocab) for tag in tags])
# total_gt = ' '.join([tag['transcription'] for tag in tags])
# eval_wer_cer(total_gt,total_pred)

In [ ]:
wers,cers = 0, 0
for tag in tags:
  wer,cer = eval_wer_cer(tag['transcription'],transcribe(tag['logits'],vocab))
  wers += wer
  cers += cer
print(wers/len(tags))
print(cers/len(tags))


KeyError: 'logits'

In [ ]:
len(tags)

1644

In [ ]:
vocab = data['vocab']
tags = data['tags']
tags = [tag for tag in tags if not re.search('\d',tag['transcription'])]

# now we will train a one fc layer network to fidd the optimal phonemic linear mapping for indian accent

class Dataset(torch.utils.data.Dataset):
  def __init__(self,tags,vocab):
    self.tags = tags
    self.max_len_logits = max([len(tag['logits']) for tag in tags])
    self.max_len_transcription = max([len(tag['transcription']) for tag in tags])
    self.vocab = vocab
    self.normalizer = EnglishTextNormalizer()

  def _pad_logits(self,x):
    return torch.nn.functional.pad(x.T,(0,self.max_len_logits-x.shape[0])).T

  def _pad_transcription(self,x):
    return torch.nn.functional.pad(x,(0,self.max_len_transcription-x.shape[0]))

  def __len__(self):
    return len(self.tags)

  def __getitem__(self,idx):
    tag = self.tags[idx]
    transcription = self.normalizer(tag['transcription'])
    len_transcription = len(transcription)
    transcription = torch.tensor([self.vocab.index(char) for char in transcription])
    transcription = self._pad_transcription(transcription)
    logits = tag['logits']
    logits = torch.tensor(logits)
    len_logits = logits.shape[0]
    logits = self._pad_logits(logits)
    return logits,transcription,len_logits,len_transcription

class PhonemeLinear(torch.nn.Module):
  def __init__(self,input_dim=29, output_dim=29):
    super().__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)

  def forward(self,x):
    x = self.linear(x)
    x = torch.nn.functional.log_softmax(x,dim=-1)
    return x

class PhonemeDoubleLinear(torch.nn.Module):
  def __init__(self,input_dim=29, output_dim=29):
    super().__init__()
    self.linear1 = torch.nn.Linear(input_dim, 512)
    self.relu = torch.nn.ReLU()
    self.linear2 = torch.nn.Linear(512, output_dim)

  def forward(self,x):
    x = self.linear1(x)
    x = self.relu(x)
    x = self.linear2(x)
    x = torch.nn.functional.log_softmax(x,dim=-1)
    return x


In [ ]:

class EmbeddingDataset(torch.utils.data.Dataset):
  def __init__(self,tags,vocab):
    self.tags = tags
    self.max_len_embedding = max([len(tag['embedding'].T) for tag in tags])
    self.max_len_transcription = max([len(tag['transcription']) for tag in tags])
    self.vocab = vocab
    self.normalizer = EnglishTextNormalizer()

  def _pad_embeddings(self,x):
    return torch.nn.functional.pad(x.T,(0,self.max_len_embedding-x.shape[0])).T

  def _pad_transcription(self,x):
    return torch.nn.functional.pad(x,(0,self.max_len_transcription-x.shape[0]))

  def __len__(self):
    return len(self.tags)

  def __getitem__(self,idx):
    tag = self.tags[idx]
    transcription = self.normalizer(tag['transcription'])
    len_transcription = len(transcription)
    transcription = torch.tensor([self.vocab.index(char) for char in transcription])
    transcription = self._pad_transcription(transcription)
    embs = tag['embedding'].T
    embs = torch.tensor(embs)
    len_embs = embs.shape[0]
    embs = self._pad_embeddings(embs)
    return embs,transcription,len_embs,len_transcription

class LinearDecoder(torch.nn.Module):
  def __init__(self,input_dim=1024, output_dim=29):
    super().__init__()
    self.linear1 = torch.nn.Linear(input_dim, output_dim)

  def forward(self,x):
    x = self.linear1(x)
    x = torch.nn.functional.log_softmax(x,dim=-1)
    return x


In [ ]:
model = LinearDecoder()
trainset = EmbeddingDataset(tags,vocab)
loader = torch.utils.data.DataLoader(trainset, batch_size=2, shuffle=True)
model(next(iter(loader))[0]).shape

torch.Size([2, 1488, 29])

In [ ]:
# model = PhonemeLinear()
# model = PhonemeDoubleLinear()
# trainset = Dataset(tags,vocab)
# loader = torch.utils.data.DataLoader(trainset, batch_size=2, shuffle=True)
# model(next(iter(loader))[0]).shape,next(iter(loader))[0].shape

In [ ]:
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt


# Define the model, optimizer, and loss function
model = LinearDecoder().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
criterion = torch.nn.CTCLoss(blank=len(vocab),zero_infinity=True)

# Create the dataset and dataloader
shuffle = np.random.permutation(len(tags))
tags = np.array(tags)[shuffle]
train_samples = int(len(tags)*0.75)
trainset = EmbeddingDataset(tags[:train_samples],vocab)
evalset = EmbeddingDataset(tags[train_samples:],vocab)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
  losses = []
  model.train()
  for logits, transcription, len_logits, len_transcription in tqdm(dataloader):
    optimizer.zero_grad()

    # Pass the logits through the model
    output = model(logits.cuda())

    # Reshape the output for CTC loss
    output = output.permute(1, 0, 2)  # (T, N, C)

    # Calculate the CTC loss
    loss = criterion(output, transcription, len_logits, len_transcription)

    # Backpropagate and optimize
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

  # evaluate wer and cer
  model.eval()
  total_pred = ''
  total_gt = ''
  total_pre_pred = ''
  eval_vocab = np.array(vocab + ['eps'])
  with torch.no_grad():
    for logits, transcription, len_logits, len_transcription in evalset:
      output = model(logits[:len_logits].unsqueeze(0).cuda()).squeeze(0).cpu()
      total_gt += ''.join(eval_vocab[transcription[:len_transcription]])
      total_pred += transcribe(output.detach().numpy(),eval_vocab)
      # if epoch == 0:
      #   total_pre_pred += transcribe(logits[:len_logits].detach().numpy(),eval_vocab)

  wer,cer = eval_wer_cer(total_gt,total_pred)
  # if epoch == 0:
  #   old_wer,old_cer = eval_wer_cer(total_gt,total_pre_pred)
  print(f"""Epoch {epoch+1}/{num_epochs}, Loss: {np.mean(losses):.3f} WER: {wer:.3f}, CER: {cer:.3f}""")

  # mat = model.linear.weight.detach().cpu().numpy()
  # plt.imshow(mat, cmap='viridis')
  # plt.colorbar()
  # plt.show()


100%|██████████| 39/39 [00:15<00:00,  2.52it/s]


Epoch 1/200, Loss: 5.070 WER: 0.998, CER: 0.981


100%|██████████| 39/39 [00:14<00:00,  2.62it/s]


Epoch 2/200, Loss: 3.378 WER: 0.982, CER: 0.921


100%|██████████| 39/39 [00:14<00:00,  2.64it/s]


...

Epoch 198/200, Loss: 0.480 WER: 0.339, CER: 0.113


100%|██████████| 39/39 [00:15<00:00,  2.58it/s]


Epoch 199/200, Loss: 0.481 WER: 0.349, CER: 0.114


100%|██████████| 39/39 [00:15<00:00,  2.49it/s]


Epoch 200/200, Loss: 0.485 WER: 0.342, CER: 0.114


In [ ]:
# project documentation
#

tensor([ 224,  160,  120,  280,  384,  600, 1024,  184,  552,  112,   48,  288,
         360,  704,  312,  304,  168,  344,  408,  200,  336,  104,   32,  232,
         584,  304,  112,  184,  216,  336,   40,  240])

In [ ]:
transcribe(logits.detach().numpy(),eval_vocab)

'it is drauft '

In [ ]:
eval_vocab[transcription[:len_transcription]]

array(['i', 't', ' ', 'a', 's', ' ', 'd', 'r', 'a', 'f', 't'], dtype='<U3')

In [ ]:
# prompt: write  an elaborate summery of the work done above

# This code snippet demonstrates a process for improving the accuracy of automatic speech recognition (ASR)
# for Indian accents using a deep learning model.

# 1. Data Preparation:
#    - The code starts by loading a dataset of audio transcriptions and corresponding logits (predicted probabilities)
#      from a pickle file.
#    - It preprocesses the data by removing samples containing numbers and normalizing the transcriptions.

# 2. Initial Evaluation:
#    - The code calculates the Word Error Rate (WER) and Character Error Rate (CER) for the initial ASR model.
#    - This provides a baseline performance metric for the model before any optimization.

# 3. Model Definition:
#    - A custom dataset class is defined to handle the input data (logits and transcriptions).
#    - The code defines two neural network models: `PhonemeLinear` (a simple linear layer) and `PhonemeDoubleLinear`
#      (a two-layer network with a ReLU activation). These models are designed to learn a linear mapping between the
#      logits and the phonemes in the transcriptions.

# 4. Training:
#    - The code sets up a training loop using PyTorch.
#    - It uses the `CTCLoss` function to calculate the loss between the model's output and the target transcriptions.
#    - The model is trained using the Adam optimizer.
#    - During training, the code tracks the loss and evaluates the WER and CER on a held-out evaluation set.

# 5. Evaluation:
#    - After each epoch, the code evaluates the model's performance on the evaluation set by calculating the WER and CER.
#    - This allows for monitoring the model's improvement over time.

# 6. Results:
#    - The code prints the loss, WER, and CER for each epoch, showing how the model's performance evolves during training.
#    - The code also compares the WER and CER of the trained model with the initial model's performance.

# In summary, this code demonstrates a basic approach to improving ASR accuracy for Indian accents by training a
# neural network to refine the phonemic mapping. The code uses a CTC loss function, which is commonly used for
# sequence-to-sequence tasks like ASR, and evaluates the model's performance using WER and CER metrics.
